# ISCA-k - Diagnóstico Completo

Este notebook testa o ISCA-k a 100% para identificar exactamente onde falha e não falha.

**Diferenças do notebook anterior:**
- Testes com missings em MÚLTIPLAS colunas (para testar PDS de verdade)
- Diagnóstico explícito do PDS (scale factors, donors rejeitados)
- Comparação directa PDS on vs off
- Análise detalhada do Adaptive K
- Testes de Phase 2

In [81]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from iscak_core import ISCAkCore
from core.adaptive_k import adaptive_k_hybrid
from core.mi_calculator import calculate_mi_mixed
from core.distances import (
    weighted_euclidean_batch, 
    weighted_euclidean_pds,
    mixed_distance_pds,
    range_normalized_mixed_distance
)
from preprocessing.type_detection import MixedDataHandler
from preprocessing.scaling import get_scaled_data, compute_range_factors

np.random.seed(42)
print("Imports OK")

Imports OK


---
## PARTE 1: Diagnóstico do PDS

O PDS (Partial Distance Strategy) permite usar donors com overlap parcial de features.
Quando o sample tem NaN em várias colunas, o PDS aplica um scale_factor para compensar.

In [83]:
def diagnose_pds(sample, donors, weights, n_features):
    """
    Diagnóstico detalhado do que o PDS está a fazer.
    
    Mostra para cada donor:
    - Overlap (quantas features em comum)
    - Scale factor aplicado
    - Distância raw vs distância scaled
    - Se foi aceite ou rejeitado
    """
    print(f"\n{'='*70}")
    print("DIAGNÓSTICO PDS")
    print(f"{'='*70}")
    
    # Contar features disponíveis no sample
    sample_avail = ~np.isnan(sample)
    n_sample_avail = sample_avail.sum()
    print(f"\nSample tem {n_sample_avail}/{n_features} features disponíveis")
    print(f"Features missing no sample: {np.where(~sample_avail)[0].tolist()}")
    
    min_overlap = max(2, n_features // 2)
    print(f"Min overlap requerido: {min_overlap} (50% de {n_features})")
    
    print(f"\n{'Donor':<8} {'Overlap':<12} {'Scale':<10} {'Dist Raw':<12} {'Dist Scaled':<12} {'Status'}")
    print("-" * 70)
    
    results = []
    for i, donor in enumerate(donors):
        # Calcular overlap
        donor_avail = ~np.isnan(donor)
        overlap_mask = sample_avail & donor_avail
        overlap = overlap_mask.sum()
        
        # Calcular distância raw (só nas features comuns)
        if overlap > 0:
            diff_sq = 0.0
            weight_sum = 0.0
            for j in range(n_features):
                if overlap_mask[j]:
                    diff = sample[j] - donor[j]
                    diff_sq += weights[j] * diff * diff
                    weight_sum += weights[j]
            dist_raw = np.sqrt(diff_sq) if weight_sum > 0 else np.inf
        else:
            dist_raw = np.inf
        
        # Scale factor e distância scaled
        if overlap >= min_overlap:
            scale_factor = np.sqrt(n_features / overlap)
            dist_scaled = dist_raw * scale_factor
            status = "✓ ACEITE"
        else:
            scale_factor = np.inf
            dist_scaled = np.inf
            status = "✗ REJEITADO (overlap insuficiente)"
        
        results.append({
            'donor_idx': i,
            'overlap': overlap,
            'scale': scale_factor,
            'dist_raw': dist_raw,
            'dist_scaled': dist_scaled,
            'accepted': overlap >= min_overlap
        })
        
        scale_str = f"{scale_factor:.3f}" if np.isfinite(scale_factor) else "N/A"
        dist_raw_str = f"{dist_raw:.4f}" if np.isfinite(dist_raw) else "inf"
        dist_scaled_str = f"{dist_scaled:.4f}" if np.isfinite(dist_scaled) else "inf"
        
        print(f"{i:<8} {overlap}/{n_features:<10} {scale_str:<10} {dist_raw_str:<12} {dist_scaled_str:<12} {status}")
    
    # Resumo
    n_accepted = sum(1 for r in results if r['accepted'])
    n_rejected = len(results) - n_accepted
    print(f"\n--- RESUMO ---")
    print(f"Donors aceites: {n_accepted}/{len(results)}")
    print(f"Donors rejeitados: {n_rejected}/{len(results)}")
    
    if n_accepted > 0:
        accepted = [r for r in results if r['accepted']]
        avg_scale = np.mean([r['scale'] for r in accepted])
        print(f"Scale factor médio (aceites): {avg_scale:.3f}")
    
    return results

### Teste PDS 1: Missings em UMA coluna (PDS não actua)

Quando só há missing numa coluna, o overlap é quase total e o scale_factor ≈ 1

In [85]:
print("="*70)
print("TESTE PDS 1: Missings em UMA coluna")
print("="*70)

# Dataset simples: 10 features, target na última
n_features = 10

# Sample: só tem missing no target (coluna 9)
sample = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, np.nan])

# Donors: todos completos
donors = np.array([
    [1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 10.0],  # Próximo
    [1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 1.2, 11.0],  # Próximo
    [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 50.0],  # Distante
])

weights = np.ones(n_features) / n_features

results = diagnose_pds(sample, donors, weights, n_features)

print(f"\n⚠️  CONCLUSÃO: Com missing em apenas 1 coluna:")
print(f"   - Overlap = {n_features-1}/{n_features} = {(n_features-1)/n_features*100:.0f}%")
print(f"   - Scale factor = sqrt({n_features}/{n_features-1}) = {np.sqrt(n_features/(n_features-1)):.3f}")
print(f"   - O PDS praticamente NÃO ACTUA!")

TESTE PDS 1: Missings em UMA coluna

DIAGNÓSTICO PDS

Sample tem 9/10 features disponíveis
Features missing no sample: [9]
Min overlap requerido: 5 (50% de 10)

Donor    Overlap      Scale      Dist Raw     Dist Scaled  Status
----------------------------------------------------------------------
0        9/10         1.054      0.0949       0.1000       ✓ ACEITE
1        9/10         1.054      0.1897       0.2000       ✓ ACEITE
2        9/10         1.054      3.7947       4.0000       ✓ ACEITE

--- RESUMO ---
Donors aceites: 3/3
Donors rejeitados: 0/3
Scale factor médio (aceites): 1.054

⚠️  CONCLUSÃO: Com missing em apenas 1 coluna:
   - Overlap = 9/10 = 90%
   - Scale factor = sqrt(10/9) = 1.054
   - O PDS praticamente NÃO ACTUA!


### Teste PDS 2: Missings em MÚLTIPLAS colunas (PDS actua)

Quando há missings em várias colunas, o scale_factor aumenta significativamente

In [87]:
print("="*70)
print("TESTE PDS 2: Missings em MÚLTIPLAS colunas")
print("="*70)

# Sample: missings em 4 colunas (cols 6, 7, 8, 9)
sample_multi = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, np.nan, np.nan, np.nan, np.nan])

# Donors com diferentes níveis de completude
donors_multi = np.array([
    [1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 1.1, 10.0],  # Completo - overlap 6/10
    [1.2, 1.2, 1.2, 1.2, 1.2, 1.2, np.nan, 1.2, 1.2, 11.0],  # Missing col 6 - overlap 5/10
    [1.3, 1.3, 1.3, 1.3, 1.3, 1.3, np.nan, np.nan, 1.3, 12.0],  # Missing cols 6,7 - overlap 4/10
    [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 50.0],  # Completo mas distante
    [1.0, 1.0, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 10.0],  # overlap 2/10 - REJEITADO
])

results = diagnose_pds(sample_multi, donors_multi, weights, n_features)

print(f"\n✓ CONCLUSÃO: Com missings em MÚLTIPLAS colunas:")
print(f"   - O PDS aplica scale factors significativos")
print(f"   - Donors com pouco overlap são REJEITADOS")
print(f"   - Isto muda completamente a selecção de vizinhos!")

TESTE PDS 2: Missings em MÚLTIPLAS colunas

DIAGNÓSTICO PDS

Sample tem 6/10 features disponíveis
Features missing no sample: [6, 7, 8, 9]
Min overlap requerido: 5 (50% de 10)

Donor    Overlap      Scale      Dist Raw     Dist Scaled  Status
----------------------------------------------------------------------
0        6/10         1.291      0.0775       0.1000       ✓ ACEITE
1        6/10         1.291      0.1549       0.2000       ✓ ACEITE
2        6/10         1.291      0.2324       0.3000       ✓ ACEITE
3        6/10         1.291      3.0984       4.0000       ✓ ACEITE
4        2/10         N/A        0.0000       inf          ✗ REJEITADO (overlap insuficiente)

--- RESUMO ---
Donors aceites: 4/5
Donors rejeitados: 1/5
Scale factor médio (aceites): 1.291

✓ CONCLUSÃO: Com missings em MÚLTIPLAS colunas:
   - O PDS aplica scale factors significativos
   - Donors com pouco overlap são REJEITADOS
   - Isto muda completamente a selecção de vizinhos!


### Teste PDS 3: Impacto do PDS na qualidade da imputação

Comparar resultados COM e SEM PDS num dataset com múltiplos missings

In [89]:
print("="*70)
print("TESTE PDS 3: Impacto na qualidade (COM vs SEM PDS)")
print("="*70)

np.random.seed(42)

# Criar dataset com 2 clusters bem separados
# Cluster 1: features ~1, targets ~10
# Cluster 2: features ~5, targets ~50

def make_cluster(center, target_center, n_samples, n_features):
    data = np.random.normal(center, 0.2, (n_samples, n_features))
    targets = np.random.normal(target_center, 1.0, n_samples)
    return np.column_stack([data, targets])

cluster1 = make_cluster(1.0, 10.0, 30, 8)
cluster2 = make_cluster(5.0, 50.0, 30, 8)
data_full = np.vstack([cluster1, cluster2])

# Criar DataFrame
cols = [f'F{i}' for i in range(8)] + ['Target']
df_complete = pd.DataFrame(data_full, columns=cols)

print(f"Dataset: {df_complete.shape[0]} linhas x {df_complete.shape[1]} colunas")
print(f"Cluster 1 (linhas 0-29): features ~1, target ~10")
print(f"Cluster 2 (linhas 30-59): features ~5, target ~50")

TESTE PDS 3: Impacto na qualidade (COM vs SEM PDS)
Dataset: 60 linhas x 9 colunas
Cluster 1 (linhas 0-29): features ~1, target ~10
Cluster 2 (linhas 30-59): features ~5, target ~50


In [90]:
# Introduzir missings em MÚLTIPLAS colunas (padrão realista)
# 20% de missings distribuídos aleatoriamente

np.random.seed(42)
df_missing = df_complete.copy()

n_total_values = df_missing.shape[0] * df_missing.shape[1]
n_missing = int(0.20 * n_total_values)  # 20% missing

# Seleccionar posições aleatórias para missings
missing_positions = []
rows = np.random.choice(60, n_missing, replace=True)
cols_idx = np.random.choice(9, n_missing, replace=True)

true_values = {}
for r, c in zip(rows, cols_idx):
    col_name = cols[c]
    if (r, col_name) not in true_values:  # Evitar duplicados
        true_values[(r, col_name)] = df_complete.loc[r, col_name]
        df_missing.loc[r, col_name] = np.nan

# Contar missings por coluna
print(f"\nMissings introduzidos: {len(true_values)}")
print(f"\nMissings por coluna:")
for col in cols:
    n_miss = df_missing[col].isna().sum()
    print(f"  {col}: {n_miss} ({n_miss/60*100:.1f}%)")

# Missings por linha (para ver padrão)
missings_per_row = df_missing.isna().sum(axis=1)
print(f"\nMissings por linha: min={missings_per_row.min()}, max={missings_per_row.max()}, média={missings_per_row.mean():.1f}")


Missings introduzidos: 92

Missings por coluna:
  F0: 16 (26.7%)
  F1: 7 (11.7%)
  F2: 11 (18.3%)
  F3: 8 (13.3%)
  F4: 11 (18.3%)
  F5: 6 (10.0%)
  F6: 14 (23.3%)
  F7: 10 (16.7%)
  Target: 9 (15.0%)

Missings por linha: min=0, max=4, média=1.5


In [91]:
# Executar COM PDS
print(f"\n{'='*30} COM PDS {'='*30}")
imputer_pds = ISCAkCore(verbose=True, use_pds=True, min_friends=3, max_friends=10)
result_pds = imputer_pds.impute(df_missing.copy(), interactive=False)

# Calcular erros
errors_pds = []
for (r, col), true_val in true_values.items():
    imputed = result_pds.loc[r, col]
    if not np.isnan(imputed):
        errors_pds.append(abs(imputed - true_val))

mae_pds = np.mean(errors_pds) if errors_pds else np.nan
print(f"\nMAE COM PDS: {mae_pds:.4f}")
print(f"Fase 2 activada: {imputer_pds.execution_stats.get('phase2_activated', False)}")


============================== COM PDS ==============================
\nCLASSIFICACAO DE VARIAVEIS:
  Numericas: 9
    ['F0', 'F1', 'F2', 'F3', 'F4']
  Binarias: 0
  Nominais: 0
  Ordinais: 0

      ISCA-k: Information-theoretic Smart Collaborative Approach      

Dataset: 60 x 9
Missings: 92 (17.0%)
Parametros: min_friends=3, max_friends=10
MI neighbors: 3
Adaptive k alpha: 0.5
Fast mode: False
FCM clustering: False
PDS (partial donors): True
  Overlap: adaptativo (maximiza por valor)
Max cycles: 3

Linhas 100% completas: 7/60 (11.7%)
Estratégia: ISCA-k+PDS primeiro, fallback se necessário

FASE 1: ISCA-k + PDS
  [1/3] Calculando Informacao Mutua...
  [2/3] Ordenando colunas por facilidade...
  [3/3] Imputando colunas...

  Resultado: 92 → 0 missings
             92 imputados (100.0%)

RESULTADO FINAL

Fases:
  ISCA-k + PDS: 92 → 0 (92 imputados, 100.0%)

✅ Fase 1 resolveu tudo (Fase 2 não necessária)

Total: 92 → 0 missings
Status: SUCESSO - Dataset 100% completo
Taxa de imputação: 

In [92]:
# Executar SEM PDS
print(f"\n{'='*30} SEM PDS {'='*30}")
imputer_no_pds = ISCAkCore(verbose=True, use_pds=False, min_friends=3, max_friends=10)
result_no_pds = imputer_no_pds.impute(df_missing.copy(), interactive=False)

# Calcular erros
errors_no_pds = []
for (r, col), true_val in true_values.items():
    imputed = result_no_pds.loc[r, col]
    if not np.isnan(imputed):
        errors_no_pds.append(abs(imputed - true_val))

mae_no_pds = np.mean(errors_no_pds) if errors_no_pds else np.nan
print(f"\nMAE SEM PDS: {mae_no_pds:.4f}")
print(f"Fase 2 activada: {imputer_no_pds.execution_stats.get('phase2_activated', False)}")


============================== SEM PDS ==============================
\nCLASSIFICACAO DE VARIAVEIS:
  Numericas: 9
    ['F0', 'F1', 'F2', 'F3', 'F4']
  Binarias: 0
  Nominais: 0
  Ordinais: 0

      ISCA-k: Information-theoretic Smart Collaborative Approach      

Dataset: 60 x 9
Missings: 92 (17.0%)
Parametros: min_friends=3, max_friends=10
MI neighbors: 3
Adaptive k alpha: 0.5
Fast mode: False
FCM clustering: False
PDS (partial donors): False
Max cycles: 3

Linhas 100% completas: 7/60 (11.7%)
Estratégia: ISCA-k clássico

FASE 1: ISCA-k
  [1/3] Calculando Informacao Mutua...
  [2/3] Ordenando colunas por facilidade...
  [3/3] Imputando colunas...

  Resultado: 92 → 0 missings
             92 imputados (100.0%)

RESULTADO FINAL

Fases:
  ISCA-k: 92 → 0 (92 imputados, 100.0%)

✅ Fase 1 resolveu tudo (Fase 2 não necessária)

Total: 92 → 0 missings
Status: SUCESSO - Dataset 100% completo
Taxa de imputação: 100.0%
Tempo total: 0.14s


MAE SEM PDS: 0.3798
Fase 2 activada: False


In [93]:
# Comparação
print(f"\n{'='*70}")
print("COMPARAÇÃO PDS vs SEM PDS")
print(f"{'='*70}")
print(f"\nMAE COM PDS:  {mae_pds:.4f}")
print(f"MAE SEM PDS:  {mae_no_pds:.4f}")

if mae_pds < mae_no_pds:
    diff = (mae_no_pds - mae_pds) / mae_no_pds * 100
    print(f"\n✓ PDS é MELHOR por {diff:.1f}%")
elif mae_no_pds < mae_pds:
    diff = (mae_pds - mae_no_pds) / mae_pds * 100
    print(f"\n⚠️  SEM PDS é MELHOR por {diff:.1f}%")
else:
    print(f"\n= Resultados iguais")

# Valores não imputados
n_nan_pds = result_pds.isna().sum().sum()
n_nan_no_pds = result_no_pds.isna().sum().sum()
print(f"\nValores NÃO imputados:")
print(f"  COM PDS: {n_nan_pds}")
print(f"  SEM PDS: {n_nan_no_pds}")


COMPARAÇÃO PDS vs SEM PDS

MAE COM PDS:  0.2019
MAE SEM PDS:  0.3798

✓ PDS é MELHOR por 46.8%

Valores NÃO imputados:
  COM PDS: 0
  SEM PDS: 0


---
## PARTE 2: Diagnóstico do Adaptive K

O Adaptive K escolhe quantos vizinhos usar baseado em:
- density_trust: quão próximos estão os vizinhos
- consistency_trust: quão concordantes são os targets dos vizinhos

In [95]:
def diagnose_adaptive_k(distances, values, min_k=3, max_k=15, alpha=0.5, is_categorical=False):
    """
    Diagnóstico detalhado do Adaptive K.
    """
    print(f"\n{'='*70}")
    print("DIAGNÓSTICO ADAPTIVE K")
    print(f"{'='*70}")
    
    print(f"\nParâmetros: min_k={min_k}, max_k={max_k}, alpha={alpha}")
    print(f"Tipo: {'Categórico' if is_categorical else 'Numérico'}")
    
    # Usar max_k vizinhos para avaliar
    k_eval = min(max_k, len(distances))
    sorted_idx = np.argsort(distances)
    closest_dist = distances[sorted_idx[:k_eval]]
    closest_vals = values[sorted_idx[:k_eval]]
    
    print(f"\nTop {k_eval} vizinhos (ordenados por distância):")
    for i in range(min(10, k_eval)):  # Mostrar top 10
        print(f"  {i+1}. dist={closest_dist[i]:.4f}, target={closest_vals[i]:.2f}")
    
    # Calcular density_trust
    mean_dist = np.mean(closest_dist[np.isfinite(closest_dist)])
    density_trust = 1.0 / (1.0 + mean_dist) if np.isfinite(mean_dist) else 0.5
    
    print(f"\n--- DENSITY TRUST ---")
    print(f"  Distância média: {mean_dist:.4f}")
    print(f"  density_trust = 1/(1+{mean_dist:.4f}) = {density_trust:.4f}")
    
    # Calcular consistency_trust
    if is_categorical:
        unique, counts = np.unique(closest_vals, return_counts=True)
        consistency_trust = counts.max() / len(closest_vals) if len(counts) > 0 else 0.5
        print(f"\n--- CONSISTENCY TRUST (Categórico) ---")
        print(f"  Classes: {dict(zip(unique, counts))}")
        print(f"  Classe dominante: {unique[np.argmax(counts)]} ({counts.max()}/{len(closest_vals)})")
        print(f"  consistency_trust = {consistency_trust:.4f}")
    else:
        mean_val = np.mean(closest_vals)
        std_val = np.std(closest_vals)
        cv = std_val / abs(mean_val) if abs(mean_val) > 1e-10 else std_val
        consistency_trust = 1.0 / (1.0 + cv)
        print(f"\n--- CONSISTENCY TRUST (Numérico) ---")
        print(f"  Média targets: {mean_val:.4f}")
        print(f"  Std targets: {std_val:.4f}")
        print(f"  CV (coef. variação) = {std_val:.4f}/{abs(mean_val):.4f} = {cv:.4f}")
        print(f"  consistency_trust = 1/(1+{cv:.4f}) = {consistency_trust:.4f}")
    
    # Calcular trust final e k
    trust = alpha * density_trust + (1 - alpha) * consistency_trust
    k = int(round(min_k + (max_k - min_k) * trust))
    k = max(min_k, min(max_k, k))
    
    print(f"\n--- K FINAL ---")
    print(f"  trust = {alpha}*{density_trust:.4f} + {1-alpha}*{consistency_trust:.4f} = {trust:.4f}")
    print(f"  k = {min_k} + ({max_k}-{min_k})*{trust:.4f} = {min_k + (max_k - min_k) * trust:.2f}")
    print(f"  k final (arredondado): {k}")
    
    # Análise de sensibilidade
    print(f"\n--- ANÁLISE DE SENSIBILIDADE ---")
    print(f"  Se trust=0.0 → k={min_k}")
    print(f"  Se trust=0.5 → k={min_k + (max_k - min_k) * 0.5:.0f}")
    print(f"  Se trust=1.0 → k={max_k}")
    print(f"  Trust actual: {trust:.4f} → k={k}")
    
    return k, trust, density_trust, consistency_trust

In [96]:
print("="*70)
print("TESTE ADAPTIVE K 1: Vizinhos CONSISTENTES")
print("="*70)

# Vizinhos próximos e com targets similares
distances_consistent = np.array([0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 1.0])
values_consistent = np.array([10.0, 10.1, 10.2, 9.9, 10.0, 10.1, 9.8, 10.2, 10.0, 10.1, 9.9, 10.0, 10.1, 10.2, 10.0])

k1, trust1, dens1, cons1 = diagnose_adaptive_k(distances_consistent, values_consistent)

print(f"\nESPERADO: k ALTO porque vizinhos concordam")

TESTE ADAPTIVE K 1: Vizinhos CONSISTENTES

DIAGNÓSTICO ADAPTIVE K

Parâmetros: min_k=3, max_k=15, alpha=0.5
Tipo: Numérico

Top 15 vizinhos (ordenados por distância):
  1. dist=0.1000, target=10.00
  2. dist=0.1500, target=10.10
  3. dist=0.2000, target=10.20
  4. dist=0.2500, target=9.90
  5. dist=0.3000, target=10.00
  6. dist=0.3500, target=10.10
  7. dist=0.4000, target=9.80
  8. dist=0.4500, target=10.20
  9. dist=0.5000, target=10.00
  10. dist=0.5500, target=10.10

--- DENSITY TRUST ---
  Distância média: 0.4833
  density_trust = 1/(1+0.4833) = 0.6742

--- CONSISTENCY TRUST (Numérico) ---
  Média targets: 10.0400
  Std targets: 0.1143
  CV (coef. variação) = 0.1143/10.0400 = 0.0114
  consistency_trust = 1/(1+0.0114) = 0.9887

--- K FINAL ---
  trust = 0.5*0.6742 + 0.5*0.9887 = 0.8315
  k = 3 + (15-3)*0.8315 = 12.98
  k final (arredondado): 13

--- ANÁLISE DE SENSIBILIDADE ---
  Se trust=0.0 → k=3
  Se trust=0.5 → k=9
  Se trust=1.0 → k=15
  Trust actual: 0.8315 → k=13

ESPERADO:

In [97]:
print("="*70)
print("TESTE ADAPTIVE K 2: Vizinhos INCONSISTENTES")
print("="*70)

# Vizinhos próximos mas com targets muito diferentes
distances_inconsistent = np.array([0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 1.0])
values_inconsistent = np.array([10.0, 50.0, 5.0, 100.0, 20.0, 80.0, 15.0, 90.0, 30.0, 70.0, 25.0, 60.0, 35.0, 55.0, 40.0])

k2, trust2, dens2, cons2 = diagnose_adaptive_k(distances_inconsistent, values_inconsistent)

print(f"\nESPERADO: k BAIXO porque vizinhos discordam")

TESTE ADAPTIVE K 2: Vizinhos INCONSISTENTES

DIAGNÓSTICO ADAPTIVE K

Parâmetros: min_k=3, max_k=15, alpha=0.5
Tipo: Numérico

Top 15 vizinhos (ordenados por distância):
  1. dist=0.1000, target=10.00
  2. dist=0.1500, target=50.00
  3. dist=0.2000, target=5.00
  4. dist=0.2500, target=100.00
  5. dist=0.3000, target=20.00
  6. dist=0.3500, target=80.00
  7. dist=0.4000, target=15.00
  8. dist=0.4500, target=90.00
  9. dist=0.5000, target=30.00
  10. dist=0.5500, target=70.00

--- DENSITY TRUST ---
  Distância média: 0.4833
  density_trust = 1/(1+0.4833) = 0.6742

--- CONSISTENCY TRUST (Numérico) ---
  Média targets: 45.6667
  Std targets: 28.6860
  CV (coef. variação) = 28.6860/45.6667 = 0.6282
  consistency_trust = 1/(1+0.6282) = 0.6142

--- K FINAL ---
  trust = 0.5*0.6742 + 0.5*0.6142 = 0.6442
  k = 3 + (15-3)*0.6442 = 10.73
  k final (arredondado): 11

--- ANÁLISE DE SENSIBILIDADE ---
  Se trust=0.0 → k=3
  Se trust=0.5 → k=9
  Se trust=1.0 → k=15
  Trust actual: 0.6442 → k=11

ESP

In [98]:
print("="*70)
print("TESTE ADAPTIVE K 3: Vizinhos DISTANTES")
print("="*70)

# Vizinhos distantes mas consistentes
distances_far = np.array([2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 7.5, 8.0, 8.5, 9.0])
values_far = np.array([10.0, 10.1, 10.2, 9.9, 10.0, 10.1, 9.8, 10.2, 10.0, 10.1, 9.9, 10.0, 10.1, 10.2, 10.0])

k3, trust3, dens3, cons3 = diagnose_adaptive_k(distances_far, values_far)

print(f"\nESPERADO: k BAIXO porque vizinhos estão longe (density_trust baixo)")

TESTE ADAPTIVE K 3: Vizinhos DISTANTES

DIAGNÓSTICO ADAPTIVE K

Parâmetros: min_k=3, max_k=15, alpha=0.5
Tipo: Numérico

Top 15 vizinhos (ordenados por distância):
  1. dist=2.0000, target=10.00
  2. dist=2.5000, target=10.10
  3. dist=3.0000, target=10.20
  4. dist=3.5000, target=9.90
  5. dist=4.0000, target=10.00
  6. dist=4.5000, target=10.10
  7. dist=5.0000, target=9.80
  8. dist=5.5000, target=10.20
  9. dist=6.0000, target=10.00
  10. dist=6.5000, target=10.10

--- DENSITY TRUST ---
  Distância média: 5.5000
  density_trust = 1/(1+5.5000) = 0.1538

--- CONSISTENCY TRUST (Numérico) ---
  Média targets: 10.0400
  Std targets: 0.1143
  CV (coef. variação) = 0.1143/10.0400 = 0.0114
  consistency_trust = 1/(1+0.0114) = 0.9887

--- K FINAL ---
  trust = 0.5*0.1538 + 0.5*0.9887 = 0.5713
  k = 3 + (15-3)*0.5713 = 9.86
  k final (arredondado): 10

--- ANÁLISE DE SENSIBILIDADE ---
  Se trust=0.0 → k=3
  Se trust=0.5 → k=9
  Se trust=1.0 → k=15
  Trust actual: 0.5713 → k=10

ESPERADO: k B

In [99]:
# Comparação
print(f"\n{'='*70}")
print("COMPARAÇÃO ADAPTIVE K")
print(f"{'='*70}")
print(f"\n{'Cenário':<25} {'Density':<12} {'Consistency':<12} {'Trust':<10} {'K'}")
print("-" * 70)
print(f"{'Consistente (próximo)':<25} {dens1:<12.4f} {cons1:<12.4f} {trust1:<10.4f} {k1}")
print(f"{'Inconsistente (próximo)':<25} {dens2:<12.4f} {cons2:<12.4f} {trust2:<10.4f} {k2}")
print(f"{'Consistente (distante)':<25} {dens3:<12.4f} {cons3:<12.4f} {trust3:<10.4f} {k3}")

print(f"\n--- ANÁLISE ---")
if k1 > k2:
    print(f"✓ k_consistente ({k1}) > k_inconsistente ({k2}) - CORRECTO")
else:
    print(f"⚠️  k_consistente ({k1}) <= k_inconsistente ({k2}) - PROBLEMA!")

print(f"\nVariação total de k: {min(k1,k2,k3)} a {max(k1,k2,k3)}")
if max(k1,k2,k3) - min(k1,k2,k3) < 3:
    print(f"⚠️  K varia pouco! A fórmula pode precisar de ajuste.")


COMPARAÇÃO ADAPTIVE K

Cenário                   Density      Consistency  Trust      K
----------------------------------------------------------------------
Consistente (próximo)     0.6742       0.9887       0.8315     13
Inconsistente (próximo)   0.6742       0.6142       0.6442     11
Consistente (distante)    0.1538       0.9887       0.5713     10

--- ANÁLISE ---
✓ k_consistente (13) > k_inconsistente (11) - CORRECTO

Variação total de k: 10 a 13


---
## PARTE 3: Diagnóstico da Phase 2

A Phase 2 é activada quando a Phase 1 não consegue imputar todos os valores.
Isto acontece quando há dependências circulares entre colunas com missings.

In [101]:
print("="*70)
print("TESTE PHASE 2: Dependências circulares")
print("="*70)

# Dataset onde cada linha tem missings em posições diferentes
# que criam dependência circular

# Base: linhas completas para referência
base_data = pd.DataFrame({
    'A': [1.0, 1.1, 1.2, 1.3, 1.4],
    'B': [2.0, 2.1, 2.2, 2.3, 2.4],
    'C': [3.0, 3.1, 3.2, 3.3, 3.4],
    'D': [4.0, 4.1, 4.2, 4.3, 4.4],
})

# Linhas com missings em padrão que força Phase 2
# Linha 5: missing em A, tem B,C,D
# Linha 6: missing em B, tem A,C,D
# Linha 7: missing em C, tem A,B,D
# Linha 8: missing em D, tem A,B,C
# Para imputar A da linha 5, preciso de donors com A preenchido
# Mas se todos os donors também têm missings...

problem_data = pd.DataFrame({
    'A': [np.nan, 1.6, 1.7, 1.8],
    'B': [2.5, np.nan, 2.7, 2.8],
    'C': [3.5, 3.6, np.nan, 3.8],
    'D': [4.5, 4.6, 4.7, np.nan],
})

df_phase2 = pd.concat([base_data, problem_data], ignore_index=True)

print(f"\nDataset:")
print(df_phase2.to_string())
print(f"\nLinhas 0-4: Completas (donors)")
print(f"Linhas 5-8: Cada uma tem 1 missing diferente")

TESTE PHASE 2: Dependências circulares

Dataset:
     A    B    C    D
0  1.0  2.0  3.0  4.0
1  1.1  2.1  3.1  4.1
2  1.2  2.2  3.2  4.2
3  1.3  2.3  3.3  4.3
4  1.4  2.4  3.4  4.4
5  NaN  2.5  3.5  4.5
6  1.6  NaN  3.6  4.6
7  1.7  2.7  NaN  4.7
8  1.8  2.8  3.8  NaN

Linhas 0-4: Completas (donors)
Linhas 5-8: Cada uma tem 1 missing diferente


In [102]:
# Executar e ver se Phase 2 activa
imputer_p2 = ISCAkCore(verbose=True, use_pds=True, min_friends=2)
result_p2 = imputer_p2.impute(df_phase2.copy(), interactive=False)

print(f"\n--- RESULTADO ---")
print(result_p2.to_string())

print(f"\n--- ESTATÍSTICAS ---")
stats = imputer_p2.execution_stats
print(f"Phase 2 activada: {stats.get('phase2_activated', 'N/A')}")
print(f"Phase 2 ciclos: {stats.get('phase2_cycles', 'N/A')}")
print(f"Phase 2 imputados: {stats.get('phase2_imputed', 'N/A')}")

\nCLASSIFICACAO DE VARIAVEIS:
  Numericas: 4
    ['A', 'B', 'C', 'D']
  Binarias: 0
  Nominais: 0
  Ordinais: 0

      ISCA-k: Information-theoretic Smart Collaborative Approach      

Dataset: 9 x 4
Missings: 4 (11.1%)
Parametros: min_friends=2, max_friends=15
MI neighbors: 3
Adaptive k alpha: 0.5
Fast mode: False
FCM clustering: False
PDS (partial donors): True
  Overlap: adaptativo (maximiza por valor)
Max cycles: 3

Linhas 100% completas: 5/9 (55.6%)
Estratégia: ISCA-k+PDS primeiro, fallback se necessário

FASE 1: ISCA-k + PDS
  [1/3] Calculando Informacao Mutua...
  [2/3] Ordenando colunas por facilidade...
  [3/3] Imputando colunas...

  Resultado: 4 → 0 missings
             4 imputados (100.0%)

RESULTADO FINAL

Fases:
  ISCA-k + PDS: 4 → 0 (4 imputados, 100.0%)

✅ Fase 1 resolveu tudo (Fase 2 não necessária)

Total: 4 → 0 missings
Status: SUCESSO - Dataset 100% completo
Taxa de imputação: 100.0%
Tempo total: 0.04s


--- RESULTADO ---
          A         B         C        D
0 

In [103]:
print("="*70)
print("TESTE PHASE 2 EXTREMO: Todos com múltiplos missings")
print("="*70)

# Dataset mais extremo: poucas linhas completas, maioria com múltiplos missings
df_extreme = pd.DataFrame({
    'A': [1.0, 1.1, np.nan, np.nan, np.nan, np.nan, 1.0, 1.1],
    'B': [2.0, 2.1, 2.2, np.nan, np.nan, 2.2, np.nan, np.nan],
    'C': [3.0, 3.1, np.nan, 3.3, np.nan, np.nan, 3.0, np.nan],
    'D': [4.0, 4.1, np.nan, np.nan, 4.4, np.nan, np.nan, 4.1],
})

print(f"\nDataset extremo:")
print(df_extreme.to_string())
print(f"\nMissings por linha: {df_extreme.isna().sum(axis=1).tolist()}")
print(f"Linhas completas: {(~df_extreme.isna().any(axis=1)).sum()}")

TESTE PHASE 2 EXTREMO: Todos com múltiplos missings

Dataset extremo:
     A    B    C    D
0  1.0  2.0  3.0  4.0
1  1.1  2.1  3.1  4.1
2  NaN  2.2  NaN  NaN
3  NaN  NaN  3.3  NaN
4  NaN  NaN  NaN  4.4
5  NaN  2.2  NaN  NaN
6  1.0  NaN  3.0  NaN
7  1.1  NaN  NaN  4.1

Missings por linha: [0, 0, 3, 3, 3, 3, 2, 2]
Linhas completas: 2


In [104]:
imputer_extreme = ISCAkCore(verbose=True, use_pds=True, min_friends=2)
result_extreme = imputer_extreme.impute(df_extreme.copy(), interactive=False)

print(f"\n--- RESULTADO ---")
print(result_extreme.to_string())

print(f"\n--- ESTATÍSTICAS ---")
stats = imputer_extreme.execution_stats
print(f"Phase 2 activada: {stats.get('phase2_activated', 'N/A')}")
print(f"Phase 2 ciclos: {stats.get('phase2_cycles', 'N/A')}")
print(f"Phase 2 imputados: {stats.get('phase2_imputed', 'N/A')}")

# Verificar se sobrou algum NaN
n_remaining_nan = result_extreme.isna().sum().sum()
print(f"\nNaN restantes: {n_remaining_nan}")

\nCLASSIFICACAO DE VARIAVEIS:
  Numericas: 4
    ['A', 'B', 'C', 'D']
  Binarias: 0
  Nominais: 0
  Ordinais: 0

      ISCA-k: Information-theoretic Smart Collaborative Approach      

Dataset: 8 x 4
Missings: 16 (50.0%)
Parametros: min_friends=2, max_friends=15
MI neighbors: 3
Adaptive k alpha: 0.5
Fast mode: False
FCM clustering: False
PDS (partial donors): True
  Overlap: adaptativo (maximiza por valor)
Max cycles: 3

Linhas 100% completas: 2/8 (25.0%)
Estratégia: ISCA-k+PDS primeiro, fallback se necessário

FASE 1: ISCA-k + PDS
  [1/3] Calculando Informacao Mutua...
  [2/3] Ordenando colunas por facilidade...
  [3/3] Imputando colunas...

  Resultado: 16 → 0 missings
             16 imputados (100.0%)

RESULTADO FINAL

Fases:
  ISCA-k + PDS: 16 → 0 (16 imputados, 100.0%)

✅ Fase 1 resolveu tudo (Fase 2 não necessária)

Total: 16 → 0 missings
Status: SUCESSO - Dataset 100% completo
Taxa de imputação: 100.0%
Tempo total: 0.04s


--- RESULTADO ---
          A         B         C      

---
## PARTE 4: Teste Integrado - Simular Benchmark Real

Testar com condições similares aos benchmarks (MCAR, MAR, MNAR)

In [106]:
def introduce_mcar(df, rate=0.2, seed=42):
    """Introduz missings MCAR (completamente aleatórios)"""
    np.random.seed(seed)
    df_missing = df.copy()
    mask = np.random.random(df.shape) < rate
    df_missing = df_missing.mask(mask)
    return df_missing

def introduce_mar(df, rate=0.2, seed=42):
    """Introduz missings MAR (dependentes de outras colunas)"""
    np.random.seed(seed)
    df_missing = df.copy()
    
    # Escolher coluna causativa (numérica)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numeric_cols) < 2:
        return introduce_mcar(df, rate, seed)
    
    cause_col = numeric_cols[0]
    affect_cols = numeric_cols[1:]
    
    # Missings mais prováveis onde cause_col é alto
    cause_vals = df[cause_col].values
    prob = (cause_vals - cause_vals.min()) / (cause_vals.max() - cause_vals.min() + 1e-10)
    prob = prob * rate * 2  # Escalar para taxa desejada
    
    for col in affect_cols:
        mask = np.random.random(len(df)) < prob
        df_missing.loc[mask, col] = np.nan
    
    return df_missing

def introduce_mnar(df, rate=0.2, seed=42):
    """Introduz missings MNAR (dependentes do próprio valor)"""
    np.random.seed(seed)
    df_missing = df.copy()
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
    for col in numeric_cols:
        vals = df[col].values
        # Missings mais prováveis para valores altos
        prob = (vals - vals.min()) / (vals.max() - vals.min() + 1e-10)
        prob = prob * rate * 2
        mask = np.random.random(len(df)) < prob
        df_missing.loc[mask, col] = np.nan
    
    return df_missing

In [107]:
print("="*70)
print("TESTE INTEGRADO: Dataset tipo benchmark")
print("="*70)

# Criar dataset com estrutura conhecida
np.random.seed(42)
n = 150

# Features correlacionadas
F1 = np.random.normal(0, 1, n)
F2 = F1 * 0.8 + np.random.normal(0, 0.5, n)  # Correlação forte com F1
F3 = np.random.normal(0, 1, n)  # Independente
F4 = F1 * 0.3 + F3 * 0.3 + np.random.normal(0, 0.7, n)  # Correlação média
Target = F1 * 2 + F2 * 1.5 + F4 * 0.5 + np.random.normal(0, 1, n)  # Target depende de F1, F2, F4

df_bench = pd.DataFrame({
    'F1': F1, 'F2': F2, 'F3': F3, 'F4': F4, 'Target': Target
})

print(f"Dataset: {df_bench.shape}")
print(f"\nCorrelações com Target:")
for col in ['F1', 'F2', 'F3', 'F4']:
    corr = np.corrcoef(df_bench[col], df_bench['Target'])[0, 1]
    print(f"  {col}: {corr:.4f}")

TESTE INTEGRADO: Dataset tipo benchmark
Dataset: (150, 5)

Correlações com Target:
  F1: 0.9113
  F2: 0.8955
  F3: 0.0304
  F4: 0.3415


In [108]:
def run_benchmark_test(df_complete, missing_type, rate, use_pds=True):
    """Executa um teste de benchmark"""
    try:
        # Cópia com índice limpo
        df_comp = df_complete.copy().reset_index(drop=True)
        
        # Introduzir missings
        if missing_type == 'MCAR':
            df_missing = introduce_mcar(df_comp, rate)
        elif missing_type == 'MAR':
            df_missing = introduce_mar(df_comp, rate)
        else:
            df_missing = introduce_mnar(df_comp, rate)
        
        df_missing = df_missing.reset_index(drop=True)
        
        # Guardar ground truth
        missing_mask = df_missing.isna()
        
        # Imputar
        imputer = ISCAkCore(verbose=False, use_pds=use_pds, min_friends=3, max_friends=10)
        result = imputer.impute(df_missing.copy(), interactive=False)
        result = result.reset_index(drop=True)
        
        # Calcular erros
        errors = []
        for col in df_comp.columns:
            for i in range(len(df_comp)):
                if missing_mask.iloc[i][col]:  # Era missing
                    true_val = df_comp.iloc[i][col]
                    imp_val = result.iloc[i][col]
                    if pd.notna(imp_val):
                        errors.append(abs(imp_val - true_val))
        
        mae = np.mean(errors) if errors else np.nan
        n_missing = missing_mask.sum().sum()
        n_imputed = len(errors)
        
        return {
            'type': missing_type,
            'rate': rate,
            'pds': use_pds,
            'n_missing': n_missing,
            'n_imputed': n_imputed,
            'mae': mae,
            'phase2': imputer.execution_stats.get('phase2_activated', False)
        }
    except Exception as e:
        print(f"ERRO em {missing_type} {rate*100:.0f}% PDS={use_pds}: {e}")
        return {
            'type': missing_type,
            'rate': rate,
            'pds': use_pds,
            'n_missing': 0,
            'n_imputed': 0,
            'mae': np.nan,
            'phase2': False
        }


In [109]:
print("="*70)
print("BENCHMARK: MCAR, MAR, MNAR x 10%, 20%, 30%, 40%")
print("="*70)

results = []

for missing_type in ['MCAR', 'MAR', 'MNAR']:
    for rate in [0.10, 0.20, 0.30, 0.40]:
        print(f"A testar {missing_type} {rate*100:.0f}%...", end=" ")
        
        # COM PDS
        r_pds = run_benchmark_test(df_bench, missing_type, rate, use_pds=True)
        results.append(r_pds)
        
        # SEM PDS
        r_no_pds = run_benchmark_test(df_bench, missing_type, rate, use_pds=False)
        results.append(r_no_pds)
        
        print(f"PDS={r_pds['mae']:.4f}, NO_PDS={r_no_pds['mae']:.4f}")

# Mostrar resultados
print(f"\n{'Type':<6} {'Rate':<6} {'PDS':<5} {'Missing':<8} {'Imputed':<8} {'MAE':<10} {'Phase2'}")
print("-" * 70)
for r in results:
    pds_str = "ON" if r['pds'] else "OFF"
    phase2_str = "YES" if r['phase2'] else "no"
    mae_str = f"{r['mae']:.4f}" if pd.notna(r['mae']) else "N/A"
    print(f"{r['type']:<6} {r['rate']*100:.0f}%   {pds_str:<5} {r['n_missing']:<8} {r['n_imputed']:<8} {mae_str:<10} {phase2_str}")


BENCHMARK: MCAR, MAR, MNAR x 10%, 20%, 30%, 40%
A testar MCAR 10%... PDS=0.7922, NO_PDS=0.7265
A testar MCAR 20%... PDS=0.9281, NO_PDS=0.8399
A testar MCAR 30%... ERRO em MCAR 30% PDS=True: single positional indexer is out-of-bounds
ERRO em MCAR 30% PDS=False: single positional indexer is out-of-bounds
PDS=nan, NO_PDS=nan
A testar MCAR 40%... ERRO em MCAR 40% PDS=True: single positional indexer is out-of-bounds
ERRO em MCAR 40% PDS=False: single positional indexer is out-of-bounds
PDS=nan, NO_PDS=nan
A testar MAR 10%... PDS=0.7775, NO_PDS=0.7357
A testar MAR 20%... PDS=0.7312, NO_PDS=0.6721
A testar MAR 30%... PDS=0.7349, NO_PDS=0.6861
A testar MAR 40%... PDS=0.8109, NO_PDS=0.7295
A testar MNAR 10%... PDS=0.8736, NO_PDS=0.7026
A testar MNAR 20%... PDS=0.9298, NO_PDS=0.6695
A testar MNAR 30%... PDS=1.0193, NO_PDS=0.8765
A testar MNAR 40%... ERRO em MNAR 40% PDS=True: single positional indexer is out-of-bounds
ERRO em MNAR 40% PDS=False: single positional indexer is out-of-bounds
PDS=nan

In [110]:
print(f"\n{'='*70}")
print("ANÁLISE: PDS vs SEM PDS")
print(f"{'='*70}")

for missing_type in ['MCAR', 'MAR', 'MNAR']:
    print(f"\n{missing_type}:")
    for rate in [0.10, 0.20, 0.30, 0.40]:
        r_pds_list = [r for r in results if r['type'] == missing_type and r['rate'] == rate and r['pds']]
        r_no_list = [r for r in results if r['type'] == missing_type and r['rate'] == rate and not r['pds']]
        
        if not r_pds_list or not r_no_list:
            print(f"  {rate*100:.0f}%: DADOS EM FALTA")
            continue
            
        r_pds = r_pds_list[0]
        r_no = r_no_list[0]
        
        if pd.isna(r_pds['mae']) or pd.isna(r_no['mae']):
            print(f"  {rate*100:.0f}%: MAE não disponível")
            continue
        
        diff = r_pds['mae'] - r_no['mae']
        winner = "PDS" if diff < 0 else "NO PDS" if diff > 0 else "EQUAL"
        
        print(f"  {rate*100:.0f}%: PDS={r_pds['mae']:.4f}, NO_PDS={r_no['mae']:.4f} → {winner} ({abs(diff):.4f})")



ANÁLISE: PDS vs SEM PDS

MCAR:
  10%: PDS=0.7922, NO_PDS=0.7265 → NO PDS (0.0657)
  20%: PDS=0.9281, NO_PDS=0.8399 → NO PDS (0.0882)
  30%: MAE não disponível
  40%: MAE não disponível

MAR:
  10%: PDS=0.7775, NO_PDS=0.7357 → NO PDS (0.0417)
  20%: PDS=0.7312, NO_PDS=0.6721 → NO PDS (0.0592)
  30%: PDS=0.7349, NO_PDS=0.6861 → NO PDS (0.0489)
  40%: PDS=0.8109, NO_PDS=0.7295 → NO PDS (0.0814)

MNAR:
  10%: PDS=0.8736, NO_PDS=0.7026 → NO PDS (0.1710)
  20%: PDS=0.9298, NO_PDS=0.6695 → NO PDS (0.2603)
  30%: PDS=1.0193, NO_PDS=0.8765 → NO PDS (0.1428)
  40%: MAE não disponível


In [111]:
print(f"\n{'='*70}")
print("ANÁLISE: Degradação com taxa de missing")
print(f"{'='*70}")

for missing_type in ['MCAR', 'MAR', 'MNAR']:
    print(f"\n{missing_type} (COM PDS):")
    maes = []
    for rate in [0.10, 0.20, 0.30, 0.40]:
        r_list = [r for r in results if r['type'] == missing_type and r['rate'] == rate and r['pds']]
        if r_list and pd.notna(r_list[0]['mae']):
            maes.append(r_list[0]['mae'])
        else:
            maes.append(np.nan)
    
    rates = [10, 20, 30, 40]
    for i in range(len(rates)-1):
        if pd.notna(maes[i]) and pd.notna(maes[i+1]) and maes[i] > 0:
            pct_increase = (maes[i+1] - maes[i]) / maes[i] * 100
            print(f"  {rates[i]}% → {rates[i+1]}%: MAE {maes[i]:.4f} → {maes[i+1]:.4f} ({pct_increase:+.1f}%)")
    
    # Verificar colapso
    if pd.notna(maes[0]) and pd.notna(maes[-1]) and maes[0] > 0:
        total_increase = (maes[-1] - maes[0]) / maes[0] * 100
        if total_increase > 200:
            print(f"  ⚠️  COLAPSO: MAE aumentou {total_increase:.0f}% de 10% para 40%")
        else:
            print(f"  ✓ Degradação controlada: {total_increase:.0f}%")



ANÁLISE: Degradação com taxa de missing

MCAR (COM PDS):
  10% → 20%: MAE 0.7922 → 0.9281 (+17.2%)

MAR (COM PDS):
  10% → 20%: MAE 0.7775 → 0.7312 (-5.9%)
  20% → 30%: MAE 0.7312 → 0.7349 (+0.5%)
  30% → 40%: MAE 0.7349 → 0.8109 (+10.3%)
  ✓ Degradação controlada: 4%

MNAR (COM PDS):
  10% → 20%: MAE 0.8736 → 0.9298 (+6.4%)
  20% → 30%: MAE 0.9298 → 1.0193 (+9.6%)


---
## PARTE 5: Diagnóstico do MI (Mutual Information)

Verificar se o MI está a capturar correctamente as relações entre variáveis

In [116]:
print("="*70)
print("DIAGNÓSTICO MI: Correlação vs MI")
print("="*70)

# Correlações
print(f"\nCorrelações de Pearson com Target:")
for col in ['F1', 'F2', 'F3', 'F4']:
    corr = np.corrcoef(df_bench[col], df_bench['Target'])[0, 1]
    print(f"  {col}: {corr:.4f}")

# Calcular MI
handler = MixedDataHandler(df_bench)
scaled_data = get_scaled_data(df_bench, handler)

mi_matrix = calculate_mi_mixed(
    encoded_data=df_bench,
    scaled_data=scaled_data,
    numeric_cols=handler.numeric_cols,
    binary_cols=handler.binary_cols,
    nominal_cols=handler.nominal_cols,
    ordinal_cols=handler.ordinal_cols
)

print(f"\nMutual Information com Target:")
for col in ['F1', 'F2', 'F3', 'F4']:
    mi = mi_matrix.loc[col, 'Target']
    print(f"  {col}: {mi:.4f}")

# Verificar ordem
corrs = {col: abs(np.corrcoef(df_bench[col], df_bench['Target'])[0, 1]) for col in ['F1', 'F2', 'F3', 'F4']}
mis = {col: mi_matrix.loc[col, 'Target'] for col in ['F1', 'F2', 'F3', 'F4']}

corr_order = sorted(corrs.keys(), key=lambda x: -corrs[x])
mi_order = sorted(mis.keys(), key=lambda x: -mis[x])

print(f"\nOrdem por |correlação|: {corr_order}")
print(f"Ordem por MI: {mi_order}")

if corr_order == mi_order:
    print(f"\n✓ MI e correlação concordam")
else:
    print(f"\n⚠️  MI e correlação discordam!")


DIAGNÓSTICO MI: Correlação vs MI

Correlações de Pearson com Target:
  F1: 0.9113
  F2: 0.8955
  F3: 0.0304
  F4: 0.3415

Mutual Information com Target:
  F1: 0.8953
  F2: 0.7635
  F3: 0.0000
  F4: 0.1284

Ordem por |correlação|: ['F1', 'F2', 'F4', 'F3']
Ordem por MI: ['F1', 'F2', 'F4', 'F3']

✓ MI e correlação concordam


---
## PARTE 6: Resumo e Conclusões

In [ ]:
print("\n" + "="*70)
print("RESUMO DO DIAGNÓSTICO COMPLETO")
print("="*70)
print("""
Execute todas as células acima e analise:

1. PDS (Partial Distance Strategy)
   - Com 1 coluna missing: PDS não actua (scale ~1)
   - Com múltiplas colunas missing: PDS aplica scale factors
   - Comparar MAE com/sem PDS para ver impacto

2. ADAPTIVE K
   - Vizinhos consistentes → k alto?
   - Vizinhos inconsistentes → k baixo?
   - Range de variação é suficiente?

3. PHASE 2
   - É activada quando há dependências circulares?
   - Quantos ciclos são necessários?
   - Sobram NaNs no final?

4. BENCHMARK (MCAR/MAR/MNAR)
   - Qual mecanismo tem pior desempenho?
   - Há colapso entre 20% e 40%?
   - PDS ajuda ou prejudica?

5. MI (Mutual Information)
   - Captura correctamente as correlações?
   - Ordem de importância está correcta?
""")